In [3]:
#Yohan's code:

import quandl
import json
import requests
from dotenv import load_dotenv

In [4]:
load_dotenv()
nasdaq_apikey = os.getenv("API_NASDAQ")


NameError: name 'os' is not defined

In [5]:
display(nasdaq_apikey)


NameError: name 'nasdaq_apikey' is not defined

In [6]:
#https://data.nasdaq.com/api/v3/datatables/ZILLOW/DATA?indicator_id=ZSFH&region_id=99999&api_key=hxzAbJQQtK44mmcH5vf3

url1="https://data.nasdaq.com/api/v3/datatables/ZILLOW/DATA?"
indicator="indicator_id=ZSFH&region_id=99999"
api_request="&api_key="
full_url = url1+indicator+api_request+nasdaq_apikey
print(full_url)

NameError: name 'nasdaq_apikey' is not defined

In [ ]:
response=requests.get(full_url)
data=response.json()
display(data)

In [ ]:
# This line is to test changes and commit.
#end of Yohan's original code. Following portions are copied from data anayltics and housing.

In [ ]:
### Installing the required packages 

!pip install pandas
!pip install quandl
!pip install matplotlib
!pip install yfinance
!pip install python-dotenv

In [ ]:
import pandas as pd 
import os
import quandl
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import date
from dotenv import load_dotenv

#yohan's addition:
import hvplot.pandas

In [ ]:
###loading the .env environment variables

load_dotenv()

In [ ]:
## getting the environment variables
q_API_KEY = os.getenv('QUANDL_API_KEY')

In [ ]:
## setting the API key
quandl.ApiConfig.api_key = q_API_KEY

In [ ]:
## getting unemployment data during the gulf war

gulf_unemp_df = quandl.get(["FRED/UNRATE", "FRED/UNEMPLOY"], trim_start = "1990-02-1", trim_end="1991-02-28", collapse = 'monthly')
gulf_unemp_df.columns = ['Gulf_unemploy_rate', 'Gulf_unemploy_level']


gulf_unemp_df

#yohan's addition:
#hvplot bar graph of gulf war unemploymment rate:
gulf_unemp_df.hvplot.bar(
    x="Date",
    y="Gulf_unemploy_rate",
).opts(
    title="Unemployment Rate in various dates during Gulf War",
    xrotation=45
)

In [ ]:
## getting unemployment data 6 months before the Russia/Ukraine Invasion to date

current_date = date.today()

rus_unemp_df = quandl.get(["FRED/UNRATE", "FRED/UNEMPLOY"], trim_start = "2021-09-1", trim_end=current_date, collapse = 'monthly')
rus_unemp_df.columns = ['rus_unemploy_rate', 'rus_unemploy_level']

rus_unemp_df


In [ ]:
## function to graph data 

def plot_graph(df, title:str, y_label: str):
    
        ## graphing the data

    cols =[col for col in df.columns]

    fig,ax = plt.subplots(1,1, figsize = (14,6))
    plt.style.use('seaborn')

    for c in cols: 
        ax.plot(c, linewidth=2, data=df)
        ax.set_title(title)
        plt.xlabel('Date')
        plt.ylabel(y_label)

        handles,labels = ax.get_legend_handles_labels()
        ax.legend(handles, labels, loc='upper left', prop = {"size":10})

    plt.show()

In [ ]:
## graph showing the unemployment trend in from 6 months before the uk-russian invasion
rus_df = rus_unemp_df[['rus_unemploy_rate']].copy()

plot_graph(rus_df, title="Graph showing the trend in unemployment rate", y_label="Unemployment rate")

# yohan's addition:
#hvplot bar graph of Ukraine War unemployment rate:
rus_df.hvplot.bar(
    x="Date",
    y="rus_unemploy_rate"
).opts(
    title="Ukraine War Unemployment Rate in Various Dates"
)

In [ ]:
### Installing the required packages 

!pip install pandas
!pip install quandl
!pip install matplotlib
!pip install yfinance
!pip install python-dotenv

In [ ]:
import pandas as pd 
import os
import quandl
import random
import matplotlib.pyplot as plt
from datetime import date
from dotenv import load_dotenv
import warnings


## this is to ignore the df.mean() and df.median() run on the ukr_rus_df
warnings.filterwarnings('ignore')

In [ ]:
###loading the .env environment variables

load_dotenv()

In [ ]:
## setting up the API key
quandl.ApiConfig.api_key = nasdaq_apikey

In [ ]:
## Using Zillow to get historical data to get data for UK_RUS housing prices 

zillow_indicators = quandl.get_table("ZILLOW/INDICATORS", paginate=True)

## Since we are looking at home values

mask = zillow_indicators['category']=='Home values'
## This df describes the indicators present in the data set. We are going to use the 
## the ZALL indicatro (ZHVI All Homes(SFR, Condo/Co-op))
home_values_ind = zillow_indicators[mask]

## getting the zillow regions available on the API
zillow_regions = quandl.get_table('ZILLOW/REGIONS', paginate= True)


In [ ]:
### All the States in the US
states = ['IA', 'KS', 'UT', 'VA', 'NC', 'NE', 'SD', 'AL', 'ID', 'FM', 'DE', 'AK', 'CT', 'PR', 'NM', 'MS', 'PW', 'CO', 'NJ', 'FL', 'MN', 'VI', 'NV', 'AZ', 'WI', 'ND', 'PA', 'OK', 'KY', 'RI', 'NH', 'MO', 'ME', 'VT', 'GA', 'GU', 'AS', 'NY', 'CA', 'HI', 'IL', 'TN', 'MA', 'OH', 'MD', 'MI', 'WY', 'WA', 'OR', 'MH', 'SC', 'IN', 'LA', 'MP', 'DC', 'MT', 'AR', 'WV', 'TX']

##  The zip region column is list of strings we will use the zip field to find the state in which
##  the region exists. 
## function to get the states in the region_field

def state_check(search_field): 
    ## separating the search field data set into a list
    search_field = [elem.strip() for elem in search_field.split(";")]
    
    for elem in search_field:
        if elem in states: 
            return elem
        
## creating a state field. Using the zip region 
mask = zillow_regions['region_type']=='zip'

zillow_regions_zip = zillow_regions[mask].copy()

## Creating a state column using the state_check function
zillow_regions_zip['state'] = zillow_regions_zip.apply(lambda x: state_check(x['region']), axis =1)

zillow_regions_zip

In [ ]:
for id, state in enumerate(states):
    
    ## getting the rows with the current state value
    mask = zillow_regions_zip['state']==state
    
    ## filtering the rows out in the dataframe
    zillow_ak = zillow_regions_zip[mask]
    
    ##getting the region_ids for that particular state
    region_ids = zillow_ak['region_id'].unique()
    
    ## to monitor the progress
    print(id)
    ## creating a list of the region ids so as to get that with those particular IDs
    ids = list(region_ids)
    
    
    
    ## Checking if ids are greater than 100 since we are only going to pick a random sample of 100
    if len(ids)>100:
        r_ids = random.sample(ids, 30)
    elif len(ids)==0:
        continue
    else: 
        r_ids = ids
    
      
    ### getting the dataframe containing the house prices using the ZALL indicator id from the home_values_ind 
    ## dataframe
    df = quandl.get_table('ZILLOW/DATA', indicator_id="ZALL", region_id=r_ids , paginate = True)
    
    ### grouping the values by date and then finding the average
    
    data = df.groupby(['date'])['value'].mean()
    
    ## Checking if it is the first iteration: 
    if id==0: 
        data_df = pd.DataFrame(data)
        data_df.rename(columns = {'value': state}, inplace = True)
        data_df.reset_index(inplace = True)
    else:
        new_df = pd.DataFrame(data)
        new_df.rename(columns = {'value': state}, inplace = True)
        new_df.reset_index(inplace = True)
        data_df = pd.merge(data_df, new_df, how = 'left')

In [ ]:
## getting recent data for the uk-rus invasion 
mask = data_df['date']>="2021-09-01"

## recent data containing average prices for every month from 6 months before the war for every state
ukr_rus_df = data_df[mask].copy()

ukr_rus_df = ukr_rus_df.round(2)

## Calculating the national average
ukr_rus_df['national_average'] = ukr_rus_df.mean(axis=1).round(2)

ukr_rus_df['national_median'] = ukr_rus_df.median(axis=1).round(2)

ukr_rus_df

In [ ]:
## plotting the national average and national median 
ukr_rus_plot_df = ukr_rus_df.set_index('date')


cols =['national_average', 'national_median']

fig,ax = plt.subplots(1,1, figsize = (14,6))
plt.style.use('seaborn')

for c in cols: 
    ax.plot(c, linewidth=2, data=ukr_rus_plot_df)
    ax.set_title('Plot of national_average and national_median')
    plt.xlabel('Date')
    plt.ylabel('Housing Prices')

    handles,labels = ax.get_legend_handles_labels()
    ax.legend(handles, labels, loc='upper left', prop = {"size":10})

plt.show()

In [ ]:
#yohan's addition. Hvplot bar graph:
ukr_rus_df.hvplot.bar(
    x="date",
    y="national_median",
).opts(
    title="Ukrainian War. Median home prices.",
    xrotation=45
)

In [ ]:
## Gulf war housing prices index. There's limited out there for this luckily FRED has quartely data for
## housing prices in the 1990s.
## Zillow housing data starts in 1996
# For some reason quandle cannot read the QUSR628BIS
## downloaded the xls filefrom here https://fred.stlouisfed.org/series/QUSR628BIS

df = pd.read_csv('Resources/gulf_war_index.csv')


df['observation_date'] = pd.to_datetime(df['observation_date'])

df.rename(columns = {'QUSR628BIS': 'quarterly_housing_prices'}, inplace = True)

In [ ]:
## gettting data from 1990-02-1 to 1991-02-28
## since it's quarterly, 

mask1 = df['observation_date'] >= "1990-01-1"

mask2 = df['observation_date'] <= "1991-03-01"

mask = mask1&mask2

gulf_df = df[mask]

gulf_df

In [ ]:
## plotting the quartely pricing index for the gulf war. 
gulf_plot_df = gulf_df.set_index('observation_date')

fig,ax = plt.subplots(1,1, figsize = (14,6))
plt.style.use('seaborn')

ax.plot('quarterly_housing_prices', linewidth=3, data=gulf_plot_df)
ax.set_title('Plot of Quarterly house pricing index during the Gulf War ')
plt.xlabel('Date')
plt.ylabel('Quarterly Housing Price Index (Index 2010=100)')

handles,labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='lower left', prop = {"size":10})

plt.show()

In [ ]:
#yohan's addition. Hvplot bar graph:
gulf_df.hvplot.bar(
    x="observation_date",
    y="quarterly_housing_prices",
).opts(
    title="Gulf War. home prices.",
    xrotation=45
)